In [3]:
!pip install -q transformers wandb


In [4]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: katrinpochtar to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import json
import random
import warnings
from functools import partial
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torchaudio
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import (
    Wav2Vec2ForSequenceClassification,
    HubertForSequenceClassification,
    Wav2Vec2FeatureExtractor,
    AutoImageProcessor,
    TimesformerForVideoClassification,
)

warnings.filterwarnings("ignore")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
METADATA = "/content/processed_data/metadata.json"
OUT_DIR = Path("/content/trained_encoders")
OUT_DIR.mkdir(parents=True, exist_ok=True)
NUM_EMOTIONS = 8

print(f"Device: {DEVICE}")




Device: cuda


In [6]:
class EmotionDataset(Dataset):
    def __init__(self, metadata_path: str, split: str, modality: str):
        with open(metadata_path) as f:
            data = json.load(f)
        self.samples = [s for s in data if s["split"] == split]
        self.modality = modality

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        item = {"emotion": s["emotion_idx"]}
        if self.modality == "audio":
            wav, _ = torchaudio.load(s["audio_path"])
            item["audio"] = wav.squeeze(0)
        elif self.modality == "video":
            frames = np.load(s["frames_path"])
            item["video"] = torch.from_numpy(frames).permute(0, 3, 1, 2).float() / 255.0
        return item


def collate_fn(batch):
    out = {"emotion": torch.tensor([b["emotion"] for b in batch])}
    if "audio" in batch[0]:
        out["audio"] = [b["audio"] for b in batch]
    if "video" in batch[0]:
        out["video"] = torch.stack([b["video"] for b in batch])
    return out


In [7]:
def crop_audio(wav, sr, duration, train):
    L = int(round(duration * sr))
    n = wav.numel()
    if n <= L:
        return torch.nn.functional.pad(wav, (0, L - n))
    start = torch.randint(0, n - L + 1, ()).item() if train else (n - L) // 2
    return wav[start:start + L]


def crop_video(video, n_frames, train):
    T = video.shape[0]
    if T <= n_frames:
        idx = torch.linspace(0, T - 1, n_frames).round().long()
        return video[idx]
    start = torch.randint(0, T - n_frames + 1, ()).item() if train else (T - n_frames) // 2
    return video[start:start + n_frames]


def prepare_audio(batch, processor, window_s, device, train=True):
    sr = 16000
    wavs = [crop_audio(a, sr, window_s, train).numpy() for a in batch["audio"]]
    enc = processor(wavs, sampling_rate=sr, return_tensors="pt", padding=True,
                    truncation=True, max_length=int(window_s * sr))
    kwargs = {"input_values": enc["input_values"].to(device)}
    if "attention_mask" in enc:
        kwargs["attention_mask"] = enc["attention_mask"].to(device)
    return kwargs, batch["emotion"].to(device)


def prepare_video(batch, processor, n_frames, device, train=True):
    clips = []
    for v in batch["video"]:
        clip = crop_video(v, n_frames, train)
        clips.append([clip[i].permute(1, 2, 0).numpy() for i in range(clip.shape[0])])
    enc = processor(clips, return_tensors="pt", do_rescale=False)
    return {"pixel_values": enc["pixel_values"].to(device)}, batch["emotion"].to(device)

In [8]:
def train_one_epoch(model, loader, prep_fn, optimizer, scaler):
    model.train()
    total_loss, preds, labels = 0.0, [], []
    for batch in tqdm(loader, leave=False):
        kwargs, y = prep_fn(batch, train=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast("cuda", enabled=DEVICE == "cuda"):
            logits = model(**kwargs).logits
            loss = nn.CrossEntropyLoss()(logits, y)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        preds.extend(logits.argmax(1).detach().cpu().tolist())
        labels.extend(y.cpu().tolist())
    return {
        "loss": total_loss / len(loader),
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }


@torch.no_grad()
def evaluate(model, loader, prep_fn):
    model.eval()
    total_loss, preds, labels = 0.0, [], []
    for batch in tqdm(loader, leave=False):
        kwargs, y = prep_fn(batch, train=False)
        with autocast("cuda", enabled=DEVICE == "cuda"):
            logits = model(**kwargs).logits
            loss = nn.CrossEntropyLoss()(logits, y)
        total_loss += loss.item()
        preds.extend(logits.argmax(1).cpu().tolist())
        labels.extend(y.cpu().tolist())
    return {
        "loss": total_loss / len(loader),
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

In [9]:
def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def run_experiment(cfg):
    seed_all(42)
    wandb.init(project="uncanny-valley-encoders", name=cfg["name"],
               group=cfg["modality"], config=cfg, reinit=True)

    modality = cfg["modality"]
    train_ds = EmotionDataset(METADATA, "train", modality)
    val_ds = EmotionDataset(METADATA, "val", modality)
    train_loader = DataLoader(train_ds, batch_size=cfg["batch_size"], shuffle=True,
                              num_workers=0, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=cfg["batch_size"], shuffle=False,
                            num_workers=0, collate_fn=collate_fn)

    if modality == "audio":
        model_cls = (HubertForSequenceClassification if "hubert" in cfg["model"].lower()
                     else Wav2Vec2ForSequenceClassification)
        model = model_cls.from_pretrained(
            cfg["model"], num_labels=NUM_EMOTIONS, ignore_mismatched_sizes=True)
        processor = Wav2Vec2FeatureExtractor.from_pretrained(cfg["model"])
        prep_fn = partial(prepare_audio, processor=processor,
                          window_s=cfg.get("window_s", 3.0), device=DEVICE)
        if hasattr(model, "freeze_feature_encoder"):
            model.freeze_feature_encoder()
    else:
        model = TimesformerForVideoClassification.from_pretrained(
            cfg["model"], num_labels=NUM_EMOTIONS, ignore_mismatched_sizes=True)
        processor = AutoImageProcessor.from_pretrained(cfg["model"])
        prep_fn = partial(prepare_video, processor=processor,
                          n_frames=cfg.get("n_frames", 8), device=DEVICE)
        for n, p in model.named_parameters():
            if "classifier" not in n:
                p.requires_grad = False

    model.to(DEVICE)
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), lr=cfg["lr"])
    scaler = GradScaler(enabled=DEVICE == "cuda")

    best_f1, patience_cnt = 0.0, 0
    save_path = OUT_DIR / cfg["name"]

    for epoch in range(cfg["epochs"]):
        # Unfreeze backbone after warmup
        if epoch == cfg.get("freeze_epochs", 2):
            for p in model.parameters():
                p.requires_grad = True
            optimizer = torch.optim.AdamW(model.parameters(), lr=cfg["lr"] * 0.1)
            scaler = GradScaler(enabled=DEVICE == "cuda")

        t = train_one_epoch(model, train_loader, prep_fn, optimizer, scaler)
        v = evaluate(model, val_loader, prep_fn)

        wandb.log({
            "epoch": epoch + 1,
            "train/loss": t["loss"], "train/acc": t["acc"], "train/f1": t["f1"],
            "val/loss": v["loss"], "val/acc": v["acc"], "val/f1": v["f1"],
            "lr": optimizer.param_groups[0]["lr"],
        })
        print(f"  [{epoch+1:2d}/{cfg['epochs']}] "
              f"t_f1={t['f1']:.3f} v_f1={v['f1']:.3f} v_loss={v['loss']:.3f}")

        if v["f1"] > best_f1:
            best_f1 = v["f1"]
            save_path.mkdir(parents=True, exist_ok=True)
            model.save_pretrained(str(save_path))
            processor.save_pretrained(str(save_path))
            patience_cnt = 0
        else:
            patience_cnt += 1
            if patience_cnt >= cfg.get("patience", 5):
                print(f"  Early stopping at epoch {epoch+1}")
                break

    wandb.log({"best_val_f1": best_f1})
    wandb.finish()
    del model
    torch.cuda.empty_cache()
    print(f"  Best F1: {best_f1:.4f} -> {save_path}\n")
    return {"name": cfg["name"], "best_f1": best_f1, "path": str(save_path)}

In [10]:
EXPERIMENTS = [
    # --- Audio: Wav2Vec2 (3 LRs x 2 window sizes) ---
    {"name": "wav2vec2-lr1e5-w3s", "modality": "audio",
     "model": "superb/wav2vec2-base-superb-er",
     "lr": 1e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "wav2vec2-lr3e5-w3s", "modality": "audio",
     "model": "superb/wav2vec2-base-superb-er",
     "lr": 3e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "wav2vec2-lr5e5-w3s", "modality": "audio",
     "model": "superb/wav2vec2-base-superb-er",
     "lr": 5e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "wav2vec2-lr3e5-w2s", "modality": "audio",
     "model": "superb/wav2vec2-base-superb-er",
     "lr": 3e-5, "window_s": 2.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    # --- Audio: HuBERT (3 LRs x 2 window sizes) ---
    {"name": "hubert-lr1e5-w3s", "modality": "audio",
     "model": "superb/hubert-base-superb-er",
     "lr": 1e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "hubert-lr3e5-w3s", "modality": "audio",
     "model": "superb/hubert-base-superb-er",
     "lr": 3e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "hubert-lr5e5-w3s", "modality": "audio",
     "model": "superb/hubert-base-superb-er",
     "lr": 5e-5, "window_s": 3.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    {"name": "hubert-lr3e5-w2s", "modality": "audio",
     "model": "superb/hubert-base-superb-er",
     "lr": 3e-5, "window_s": 2.0, "batch_size": 8,
     "epochs": 25, "freeze_epochs": 3, "patience": 5},

    # --- Video: TimeSformer (2 LRs x 2 frame counts + freeze variant) ---
    {"name": "timesformer-lr1e5-8f", "modality": "video",
     "model": "facebook/timesformer-base-finetuned-k400",
     "lr": 1e-5, "n_frames": 8, "batch_size": 4,
     "epochs": 15, "freeze_epochs": 1, "patience": 4},

    {"name": "timesformer-lr3e5-8f", "modality": "video",
     "model": "facebook/timesformer-base-finetuned-k400",
     "lr": 3e-5, "n_frames": 8, "batch_size": 4,
     "epochs": 15, "freeze_epochs": 1, "patience": 4},

    {"name": "timesformer-lr1e5-16f", "modality": "video",
     "model": "facebook/timesformer-base-finetuned-k400",
     "lr": 1e-5, "n_frames": 16, "batch_size": 2,
     "epochs": 15, "freeze_epochs": 1, "patience": 4},

    {"name": "timesformer-lr3e5-16f", "modality": "video",
     "model": "facebook/timesformer-base-finetuned-k400",
     "lr": 3e-5, "n_frames": 16, "batch_size": 2,
     "epochs": 15, "freeze_epochs": 1, "patience": 4},

    {"name": "timesformer-lr1e5-8f-freeze3", "modality": "video",
     "model": "facebook/timesformer-base-finetuned-k400",
     "lr": 1e-5, "n_frames": 8, "batch_size": 4,
     "epochs": 15, "freeze_epochs": 3, "patience": 4},
]

results = []
for exp in EXPERIMENTS:
    print(f"{'='*60}\n{exp['name']}\n{'='*60}")
    results.append(run_experiment(exp))

wav2vec2-lr1e5-w3s


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-base-superb-er and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 256]) in the checkpoint and torch.Size([8, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  [13/15] t_f1=0.844 v_f1=0.419 v_loss=1.473


  [14/15] t_f1=0.872 v_f1=0.464 v_loss=1.415


  [15/15] t_f1=0.862 v_f1=0.460 v_loss=1.372


best_val_f1,▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
lr,███▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▁▁▂▄▅▆▆▇▇▇▇███
train/f1,▁▁▂▃▄▅▆▆▇▇▇████
train/loss,██▇▇▆▅▄▃▃▂▂▂▁▁▁
val/acc,▁▁▂▄▅▅▆▇▆▇▇▇▇██
val/f1,▁▂▂▄▄▅▅▇▆▇▇▇▇██
val/loss,██▇▅▄▃▃▂▃▁▂▁▂▁▁
best_val_f1,0.46417
epoch,15


  Best F1: 0.4642 -> /content/trained_encoders/timesformer-lr1e5-8f-freeze3



In [11]:
print(f"\n{'='*60}")
print("RESULTS SUMMARY")
print(f"{'='*60}")
print(f"{'Name':30s} {'Best Val F1':>12s}")
print("-" * 44)
for r in sorted(results, key=lambda x: -x["best_f1"]):
    print(f"{r['name']:30s} {r['best_f1']:12.4f}")


RESULTS SUMMARY
Name                            Best Val F1
--------------------------------------------
hubert-lr5e5-w3s                     0.6239
wav2vec2-lr5e5-w3s                   0.6192
timesformer-lr3e5-16f                0.5897
wav2vec2-lr3e5-w3s                   0.5635
timesformer-lr1e5-16f                0.5461
hubert-lr3e5-w3s                     0.5376
timesformer-lr3e5-8f                 0.5093
timesformer-lr1e5-8f                 0.4811
timesformer-lr1e5-8f-freeze3         0.4642
wav2vec2-lr3e5-w2s                   0.4113
wav2vec2-lr1e5-w3s                   0.3624
hubert-lr1e5-w3s                     0.2895
hubert-lr3e5-w2s                     0.2327


In [12]:
import torch
torch.cuda.empty_cache()
print("GPU RAM cleaned.")

GPU RAM cleaned.


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  17536 KiB |   7016 MiB | 740660 GiB | 740660 GiB |
|       from large pool |  17536 KiB |   7010 MiB | 734773 GiB | 734773 GiB |
|       from small pool |      0 KiB |    111 MiB |   5886 GiB |   5886 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  17536 KiB |   7016 MiB | 740660 GiB | 740660 GiB |
|       from large pool |  17536 KiB |   7010 MiB | 734773 GiB |